<a href="https://colab.research.google.com/github/davidcpage/Imagenette-experiments/blob/master/Imagenette_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The aim of this notebook is to convert the Imagenette/woof training examples from https://github.com/lessw2020/Ranger-Mish-ImageWoof-5 to the new fastai v2 codebase. 

### Setup

Install fastai_dev (aka v2) You may need to restart after installing pillow to pick up the new version.

In [0]:
!python -m pip install typeguard
!python -m pip install --upgrade pillow
!git clone -q https://github.com/fastai/fastai_dev.git

RANGER = 'https://raw.githubusercontent.com/lessw2020/Ranger-Deep-Learning-Optimizer/master/ranger.py'
MXRESNET = 'https://raw.githubusercontent.com/lessw2020/Ranger-Mish-ImageWoof-5/master/mxresnet.py'
!wget -q $RANGER -O fastai_dev/dev/ranger.py
!wget -q $MXRESNET -O fastai_dev/dev/mxresnet.py

Install Nvidia DALI, which we use for fast dataloading/augmentation below.

In [0]:
!python -m pip install --extra-index-url https://developer.download.nvidia.com/compute/redist/cuda/10.0 nvidia-dali

Change to the fastai_dev directory, basic imports and device setup.

In [1]:
%cd fastai_dev/dev

from functools import partial
import numpy as np
import torch
import torch.nn as nn

device = torch.device(torch.cuda.current_device())
torch.backends.cudnn.benchmark = True

/home/jupyter/fastai_dev/dev


#### Params

In [0]:
size = 128
bs = 64

random_aspect_ratio = (3/4, 4/3)
random_area = (0.35, 1.)
val_xtra_size = 32 
interpolation = 2 #FIXME: what is this?

data_url = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette'

### Fastai v1 training

First let's establish a baseline on the v1 codebase. The code here is essentially taken from https://github.com/lessw2020/Ranger-Mish-ImageWoof-5/blob/master/train.py. 

**NB:** throughout the notebook we use fully qualified names for imported functions to avoid name-clashes between the two fastai codebases.

In [4]:
import fastai.datasets, fastai.vision, fastai.vision.data, fastai.basic_train
import fastai.metrics, fastai.layers, fastai.callbacks

import ranger
import mxresnet

Mish activation loaded...


In [0]:
source = fastai.datasets.untar_data(data_url)

data = (fastai.vision.data.ImageList.from_folder(source).split_by_folder(valid='val')
            .label_from_folder().transform(([fastai.vision.flip_lr(p=0.5)], []), size=size)
            .databunch(bs=bs, num_workers=4)
            .presize(size, scale=random_area, ratio=random_aspect_ratio, val_xtra_size=val_xtra_size, interpolation=interpolation)
            .normalize(fastai.vision.imagenet_stats))

In [0]:
def flat_then_cosine_sched(learn, n_batch, lr, annealing_start):
    return fastai.callbacks.GeneralScheduler(learn, phases=[
        fastai.callbacks.TrainingPhase(annealing_start*n_batch).schedule_hp('lr', lr),
        fastai.callbacks.TrainingPhase((1-annealing_start)*n_batch).schedule_hp('lr', lr, anneal=fastai.callback.annealing_cos)     
    ])

def train(
    data, 
    num_epoch=5,
    model_func=partial(mxresnet.mxresnet50, c_out=10, sa=1, sym=0),
    opt_func=partial(ranger.Ranger, betas=(0.95, 0.99), eps=1e-6),
    lr_schedule=partial(flat_then_cosine_sched, lr=4e-3, annealing_start=0.72),
    wd=1e-2,
):
    learn = fastai.basic_train.Learner(data, model_func(), wd=wd, opt_func=opt_func,
                metrics=[fastai.metrics.accuracy],
                bn_wd=False, true_wd=True,
                loss_func = fastai.layers.LabelSmoothingCrossEntropy(),
                callback_fns=[]).to_fp16(dynamic=True)
    learn.fit(num_epoch, callbacks=[
        lr_schedule(learn, n_batch=len(learn.data.train_dl) * num_epoch)])
    return learn

In order to keep things relatively fast, let's train an mxresnet18 model for 5 epochs:

In [7]:
learn = train(data, 5, model_func=partial(mxresnet.mxresnet18, c_out=10, sa=1, sym=0))

epoch,train_loss,valid_loss,accuracy,time
0,1.442921,1.328564,0.670000,00:35
1,1.207998,1.043967,0.802000,00:34
2,1.109578,1.150266,0.736000,00:34
3,1.045805,0.926282,0.836000,00:34
4,0.915382,0.844332,0.886000,00:34


### Nvidia DALI

We can speed things up (especially for small models) by using Nvidia DALI to do the dataloading/augmentation. For now we will use this for both fastai v1 and v2 models although we will want to test v2 dataloading at the end. The details of the code are not very interesting.


In [0]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator
import math

class _Pipe(Pipeline):
    def __init__(self, graph, batch_size, num_threads, device_id, seed):
        super().__init__(batch_size, num_threads, device_id, seed=seed)
        self.define_graph = graph
        self.build()

class DALIDataLoader():
    def __init__(self, graph, batch_size, drop_last=False, num_threads=4, device=None, seed=-1):
        self.device = device
        self.pipe = _Pipe(graph, batch_size, num_threads=num_threads, device_id=self.device.index, seed=seed)
        n = self.pipe.epoch_size('Reader')
        if drop_last:
            self.length = n // batch_size
            n = self.length * batch_size 
        else:
            self.length = math.ceil(n//batch_size)
        self.dali_iter = DALIGenericIterator([self.pipe], ['data', 'label'], n, auto_reset=True, fill_last_batch=False)

    def __iter__(self): return ((batch[0]['data'], 
                                 batch[0]['label'].squeeze().to(dtype=torch.int64, device=self.device)
                                ) for batch in self.dali_iter)
    
    def __len__(self): return self.length

def train_graph(data_dir, size, random_aspect_ratio, random_area, interp_type,
                stats=fastai.vision.imagenet_stats):
    inputs = ops.FileReader(file_root=data_dir, random_shuffle=True)
    decode = ops.ImageDecoderRandomCrop(
            device='mixed', output_type=types.RGB, 
            random_aspect_ratio=random_aspect_ratio, random_area=random_area, num_attempts=100)
    resize = ops.Resize(device='gpu', resize_x=size, resize_y=size, 
                        interp_type=interp_type)
    mean, std = [[x*255 for x in stat] for stat in stats]
    crop_mirror_norm = ops.CropMirrorNormalize(
                        device='gpu', output_dtype=types.FLOAT16, output_layout=types.NCHW, 
                        crop=(size, size), image_type=types.RGB, mean=mean, std=std)
    coin = ops.CoinFlip(probability=0.5)

    def define_graph():    
        jpegs, labels = inputs(name='Reader')
        output = crop_mirror_norm(resize(decode(jpegs)), mirror=coin())
        return [output, labels]
    return define_graph

def valid_graph(data_dir, size, val_xtra_size, interp_type, stats=fastai.vision.imagenet_stats, mirror=0):
    inputs = ops.FileReader(file_root=data_dir, random_shuffle=False)
    decode = ops.ImageDecoder(device='mixed', output_type=types.RGB)
    resize = ops.Resize(device='gpu', resize_shorter=size+val_xtra_size, 
                        interp_type=interp_type)
    mean, std = [[x*255 for x in stat] for stat in stats]
    crop_mirror_norm = ops.CropMirrorNormalize(
                        device='gpu', output_dtype=types.FLOAT16, output_layout=types.NCHW, 
                        crop=(size, size), image_type=types.RGB, mean=mean, std=std, mirror=mirror)
    def define_graph():
        jpegs, labels = inputs(name='Reader')
        output = crop_mirror_norm(resize(decode(jpegs)))
        return [output, labels]
    return define_graph

class MockV1DataBunch():
    def __init__(self, train_dl, valid_dl, path='dummy', empty_val=False):
        self.train_dl = train_dl
        if not hasattr(train_dl, 'dataset'): train_dl.dataset = 'dummy'
        self.valid_dl = valid_dl
        if not hasattr(valid_dl, 'dataset'): valid_dl.dataset = 'dummy'
        self.path = path
        self.device = train_dl.device
        self.empty_val = empty_val
    def add_tfm(self, tfm): pass
    def remove_tfm(self, tfm): pass

The validation set of Imagenette is tiny, consisting of 500 examples only. This leads to substantial noise in the validation accuracies. To help a little, we are going to concatenate a left-right flipped version onto to the validation set to get an effective 1000 examples:

In [0]:
interp_type = types.INTERP_TRIANGULAR

from itertools import chain
class Chain():
    def __init__(self, *dls):
        self.dls = dls
        self.device = self.dls[0].device
    def __iter__(self): return chain(*self.dls)
    def __len__(self): return sum(len(dl) for dl in self.dls)

train_dl = lambda seed=-1: DALIDataLoader(train_graph(source/'train', size, random_aspect_ratio, random_area, interp_type), bs, drop_last=True, device=device, seed=seed)
valid_dl = lambda: Chain(
        DALIDataLoader(valid_graph(source/'val', size, val_xtra_size, interp_type), bs, drop_last=False, device=device),
        DALIDataLoader(valid_graph(source/'val', size, val_xtra_size, interp_type, mirror=1), bs, drop_last=False, device=device),
    )

Now we are ready to test training on the v1 codebase with DALI dataloading:

In [10]:
dali_data = MockV1DataBunch(train_dl(), valid_dl())
learn = train(dali_data, 5, model_func=partial(mxresnet.mxresnet18, c_out=10, sa=1, sym=0))

epoch,train_loss,valid_loss,accuracy,time
0,1.465935,1.290174,0.696833,00:19
1,1.205533,0.995054,0.813406,00:18
2,1.100494,1.030867,0.805430,00:18
3,1.036184,0.904659,0.846014,00:18
4,0.915965,0.843804,0.877828,00:19


The training is substantially faster (at least on a GCP T4 GPU with 4 cpu cores) and accuracy is similar (although noise is still very large.)

### Fastai v2 model

Next we want to compare a model using the v2 codebase to the v1 model above. Here is the v1 model again:

In [0]:
model_v1 = partial(mxresnet.mxresnet18, c_out=10, sa=1, sym=0)

A similar model is available in v2. To start with let's use the Mish activation class from the v1 model to minimise differences.

In [12]:
import local.vision.models.xresnet
mish = mxresnet.Mish()
model_v2 = partial(local.vision.models.xresnet.xresnet18, c_out=10, sa=1, sym=0, act_cls=(lambda: mish))

Mish activation loaded...


First let's compare the types of modules in the two models:

In [13]:
s1 = set(type(x) for x in model_v1().modules())
s2 = set(type(x) for x in model_v2().modules())
s1^s2 #symmetric difference

{local.layers.ConvLayer,
 local.layers.Flatten,
 local.layers.ResBlock,
 local.vision.models.xresnet.XResNet,
 mxresnet.Flatten,
 mxresnet.MXResNet,
 mxresnet.ResBlock,
 mxresnet.SimpleSelfAttention,
 torch.nn.modules.activation.ReLU,
 torch.nn.modules.conv.Conv1d}

Some of these are implementation specific compound layers. Let's ignore them for now and we will come back to them later if necessary.

In [0]:
types_to_ignore = {
    torch.nn.Sequential,

    mxresnet.Flatten, 
    mxresnet.MXResNet, 
    mxresnet.ResBlock, 

    local.layers.Flatten, 
    local.vision.models.xresnet.XResNet,
    local.layers.ResBlock, 
    local.layers.ConvLayer, 
}

filtered_modules = lambda model: (x for x in model.modules() if type(x) not in types_to_ignore)

Now let's compare the filtered modules in a bit more detail by comparing `repr` strings.

In [15]:
s1 = set(repr(x) for x in filtered_modules(model_v1()))
s2 = set(repr(x) for x in filtered_modules(model_v2()))
s1 ^ s2

{'Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)',
 'Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)',
 'ReLU()',
 'SimpleSelfAttention(\n  (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)\n)'}

Let's locate the offending modules in the v1 model:

In [16]:
{k: x for k,x in model_v1().named_modules() if repr(x) in s1^s2}

{'4.1.sa': SimpleSelfAttention(
   (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
 ), '4.1.sa.conv': Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)}

and in the v2 model:

In [17]:
{k: x for k,x in model_v2().named_modules() if repr(x) in s1^s2}

{'0.2': ReLU(),
 '1.0': Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 '1.2': ReLU(),
 '2.2': ReLU()}

The v1 model has a SimpleSelfAttention layer which is missing from the v2 model, whilst the v2 model still has some ReLU activations at early layers and one Conv layer has a different shape. Let's fix these issues with a modified v2 model:

In [0]:
class XResNet(nn.Sequential):
    def __init__(self, expansion, layers, c_in=3, c_out=1000, 
                 sa=False, sym=False, act_cls=local.basics.defaults.activation,
                 ):
        stem = []
        sizes = [c_in, 16,32,64] if c_in < 3 else [c_in, 32, 64, 64] 
        for i in range(3):
            stem.append(local.layers.ConvLayer(sizes[i], sizes[i+1], stride=2 if i==0 else 1, act_cls=act_cls))

        block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
        blocks = [self._make_layer(expansion, ni=block_szs[i], nf=block_szs[i+1], blocks=l, stride=1 if i==0 else 2,
                                  sa=sa if i==len(layers)-4 else False, sym=sym, act_cls=act_cls)
                  for i,l in enumerate(layers)]
        super().__init__(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *blocks,
            nn.AdaptiveAvgPool2d(1), local.layers.Flatten(),
            nn.Linear(block_szs[-1]*expansion, c_out),
        )
        local.vision.models.xresnet.init_cnn(self)

    def _make_layer(self, expansion, ni, nf, blocks, stride, sa, sym, act_cls):
        return nn.Sequential(
            *[local.layers.ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1,
                      sa if i==(blocks-1) else False, sym=sym, act_cls=act_cls)
              for i in range(blocks)])
        
xresnet18 = partial(XResNet, expansion=1, layers=[2,2,2,2])
xresnet50 = partial(XResNet, expansion=4, layers=[3,4,6,3])

Let's instantiate the new model and run our check from before to compare with the v1 model:

In [0]:
model_v2b = partial(xresnet18, c_out=10, sa=1, sym=0, act_cls=(lambda: mish))

In [20]:
types_to_ignore.add(XResNet)

s1 = set(repr(x) for x in filtered_modules(model_v1()))
s2 = set(repr(x) for x in filtered_modules(model_v2b()))
s1 ^ s2

set()

Great. Next we'd like to check that the forward computation of the two models ties out. The difficulty with this is that initialisation calls random number generators and even if we fix random seeds beforehand, any difference in the sequence of random calls for the two models will lead to divergence.

We can check that other details of the forward computation agree by attempting to set the same initialisation for both models:

In [0]:
def reset_dummy_init(model, seed=123):
    for m in filtered_modules(model):
        if hasattr(m, 'reset_parameters'):
            torch.manual_seed(seed)
            m.reset_parameters()
    return model

def compare_fwd(model1, model2):
    random_batch = torch.randn(bs,3,size,size, device=device)
    assert np.allclose(
        model1.to(device)(random_batch).detach().cpu().numpy(), 
        model2.to(device)(random_batch).detach().cpu().numpy()
    )
    return True

In [22]:
compare_fwd(
    reset_dummy_init(model_v1()),
    reset_dummy_init(model_v2b()),
)

True

This is promising. It remains to check that the initialisations of the two models agree. In general this can be somewhat tricky for the reasons given above. The current situation is actually much nicer as both models are initialised with a final call to an `init_cnn` function and if we fix the random seed before this call we get agreement:

In [23]:
model1 = model_v1()
torch.manual_seed(123)
mxresnet.init_cnn(model1)

model2 = model_v2b()
torch.manual_seed(123)
local.vision.models.xresnet.init_cnn(model2)

compare_fwd(model1, model2)

True

### Fastai v2 training

Here is a first attempt at training using the v2 model and codebase + the DALI dataloader we used above. We will use the ranger optimiser from the v1 codebase for now:

In [0]:
import local.callback.all #need to import this to patch local.basics.Learner with .to_fp16() method
RangerWrapper = lambda *args, **kwargs: local.basics.OptimWrapper(ranger.Ranger(*args, **kwargs))

In [25]:
dali_data = local.basics.DataBunch(train_dl(), valid_dl())

learn = local.basics.Learner(
            dali_data, 
            xresnet18(c_out=10, sa=1, sym=0, act_cls=(lambda: mish)), 
            metrics=[local.basics.accuracy],
            loss_func=local.basics.LabelSmoothingCrossEntropy(), 
            lr=4e-3,
            opt_func=partial(RangerWrapper, betas=(0.95, 0.99), eps=1e-6)
        ).to_fp16() 
learn.fit_flat_cos(5, pct_start=0.72)

epoch,train_loss,valid_loss,accuracy,time
0,1.465373,1.252990,0.707000,00:20
1,1.199134,1.123996,0.760000,00:20
2,1.107051,1.133414,0.758000,00:20
3,1.030232,0.959145,0.826000,00:20
4,0.917346,0.838301,0.886000,00:20


To be continued...